In [25]:
import os
import json
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA
from langchain.schema import Document

In [26]:
with open("../data/insurance_policies.json", "r") as f:
    data = json.load(f)

In [27]:

docs = [
    Document(
        page_content=(
            f"{d['title']}\n"
            f"Type: {d['type']}\n"
            f"Max Benefit: {d['max_benefit']}\n"
            f"Coverage: {', '.join(d['coverage'])}\n"
            f"Conditions: {', '.join(d['conditions'])}\n"
        ),
        metadata={"source": d["id"]}
    )
    for d in data
]

In [28]:
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(
    documents=docs,
    embedding=embedding_model,
    persist_directory="../chroma_store"
)
vectorstore.persist()

In [29]:
os.environ["GOOGLE_API_KEY"] = "api key"

In [30]:
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
    chain_type="stuff"
)


In [31]:
query = "What is the eligibility for international travel insurance?"
response = qa_chain.run(query)

In [32]:
print(response)

The international travel insurance is available to travelers aged 0 to 70 undertaking international journeys. All applicants must submit valid identity and income proofs, and medical screening may be required based on age and health history.


In [35]:
print(qa_chain.run("maximum value i can avail from travel insurance"))

The maximum benefit you can avail from the travel insurance plans listed is ₹15,00,000.


_IncompleteInputError: incomplete input (2413337040.py, line 1)

In [38]:
print(qa_chain.run("give me all the details about the health insurance"))

Comprehensive Health Insurance (Policy Number: 1075):
*   **Coverage Overview:** Hospitalization, outpatient care, maternity, and emergency services. Designed to offer financial protection against unforeseen events and liabilities.
*   **Eligibility Criteria:** Individuals aged 18 to 65 and dependent children up to age 25. Requires valid identity and income proofs. Medical screening may be required based on age and health history.
*   **Policy Terms:** Subject to sub-limits and co-payments as detailed in the policy schedule. Pre-existing diseases are covered after a waiting period of 36 months.
*   **Renewal & Cancellation:** Must be renewed annually before the due date. The insurer reserves the right to cancel the policy in case of non-payment of premium, fraud, or violation of policy terms.

CareSecure Health Plus: (Plan ID: 1013 & 1014)
*   **Category:** Health Insurance
*   **Coverage Details:** In-patient hospitalization expenses; pre and post hospitalization (30 and 60 days respe

In [37]:
print(qa_chain.run("give me more information regarding this policy"))

Could you please specify which policy you'd like more information on? There are three policies listed:

*   **Standard Homeowners Insurance (Policy Number: 1049)**
*   **Comprehensive Health Insurance (Policy Number: 1003)**
*   **Standard Homeowners Insurance (Policy Number: 1042)**

Once you specify which policy, I can provide more details.
